<a href="https://colab.research.google.com/github/ancgate/hvfhv_weather_analysis/blob/main/python_scripts/extract_hvfhv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Libraries


In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install azure-storage-blob
!pip install google-cloud-storage
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.6/405.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 6.4 MB/s eta 0:00:00


## Importing Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import json
import re
import os
import time
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO

# Extracting Data from TLC Website

In [ ]:
AZURE_STORAGE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=sthvfhv;AccountKey=s80KlsohKqOOC6bLjRJgpmJiPqylM82RIRgREa8EAl0pttAEX9xjGZ1UFzKP6McqTpnMtcvI8mCq+AStptDIcw==;EndpointSuffix=core.windows.net"
CONTAINER_NAME = "hvfhv"

file_urls = [ "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet",
              "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-02.parquet",
              "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-03.parquet",
              "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-04.parquet",
              "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-05.parquet",
              "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-06.parquet",
              "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-07.parquet",
              "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-08.parquet",
]

blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)


def download_and_upload(url):
    filename = url.split("/")[-1]

    # Download the file
    print(f"Downloading {filename}...")
    response = requests.get(url)
    response.raise_for_status()

    # Upload to Azure Blob Storage
    blob_client = container_client.get_blob_client(filename)
    print(f"Uploading {filename} to Azure Blob Storage...")
    blob_client.upload_blob(response.content, overwrite=True)

In [ ]:
for url in file_urls:
    download_and_upload(url)

Uploading fhvhv_tripdata_2024-01.parquet to Azure Blob Storage...
Uploading fhvhv_tripdata_2024-02.parquet to Azure Blob Storage...
Uploading fhvhv_tripdata_2024-03.parquet to Azure Blob Storage...
Uploading fhvhv_tripdata_2024-04.parquet to Azure Blob Storage...
Uploading fhvhv_tripdata_2024-05.parquet to Azure Blob Storage...
Uploading fhvhv_tripdata_2024-06.parquet to Azure Blob Storage...
Uploading fhvhv_tripdata_2024-07.parquet to Azure Blob Storage...
Uploading fhvhv_tripdata_2024-08.parquet to Azure Blob Storage...


# Extracting Data from OpenMeteo

In [ ]:
lookup_df = pd.read_csv('lookup.csv')
lookup_df.head()

,Unnamed: 0,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,latitude,longitude,zipcode
0,0,1,0.116357,0.000782,Newark Airport,1,EWR,40.689488,-74.171526,07114
1,1,2,0.433470,0.004866,Jamaica Bay,2,Queens,40.610791,-73.822490,11693
2,2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,40.865745,-73.844947,10469
3,3,4,0.043567,0.000112,Alphabet City,4,Manhattan,40.724137,-73.977726,10009
4,4,5,0.092146,0.000498,Arden Heights,5,Staten Island,40.550665,-74.187537,10312


In [ ]:
combined_weather_df = pd.DataFrame()
for index, row in lookup_df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    print(latitude, longitude)
    OPEN_METEO_URL = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,precipitation,rain,snowfall,snow_depth&start_date=2024-01-01&end_date=2024-08-31&timezone=EST'
    response = requests.get(OPEN_METEO_URL)
    data = response.json()
    print(data)
    # convert to dataframe
    df = pd.DataFrame(data['hourly'])
    # append to dataframe
    combined_weather_df = pd.concat([combined_weather_df, df])
combined_weather_df.head()

40.689488139614994 -74.17152567596138
{'latitude': 40.69207, 'longitude': -74.18307, 'generationtime_ms': 213.00005912780762, 'utc_offset_seconds': -18000, 'timezone': 'America/New_York', 'timezone_abbreviation': 'EST', 'elevation': 1.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'precipitation': 'mm', 'rain': 'mm', 'snowfall': 'cm', 'snow_depth': 'm'}, 'hourly': {'time': ['2024-01-01T00:00', '2024-01-01T01:00', '2024-01-01T02:00', '2024-01-01T03:00', '2024-01-01T04:00', '2024-01-01T05:00', '2024-01-01T06:00', '2024-01-01T07:00', '2024-01-01T08:00', '2024-01-01T09:00', '2024-01-01T10:00', '2024-01-01T11:00', '2024-01-01T12:00', '2024-01-01T13:00', '2024-01-01T14:00', '2024-01-01T15:00', '2024-01-01T16:00', '2024-01-01T17:00', '2024-01-01T18:00', '2024-01-01T19:00', '2024-01-01T20:00', '2024-01-01T21:00', '2024-01-01T22:00', '2024-01-01T23:00', '2024-01-02T00:00', '2024-01-02T01:00', '2024-01-02T02:00', '2024-01-02T03:00', '2024-01-02T04:00', '2024-01-02T05:00', '2024-0

In [ ]:
csv_buffer = StringIO()
combined_df.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)
blob_name = "weather_data.csv"
# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(csv_buffer.getvalue(), blob_type="BlockBlob", overwrite=True)

print("CSV uploaded to Azure Blob Storage successfully.")

## Extracting data from Open Data NY using API

In [ ]:
6225611 /50000

124.51222

In [ ]:
for i in range(1,126,1):
  URL = f'https://data.cityofnewyork.us/resource/k397-673e.json?$limit=50000&$offset={i}'
  response = requests.get(URL)
  data = response.json()
  print(len(data))

In [ ]:
URL = f'https://data.cityofnewyork.us/api/views/k397-673e/rows.csv?fourfour=k397-673e&cacheBust=1730300401&date=20241105&accessType=DOWNLOAD'
response = requests.get(URL)
data = response.json()
print(len(data))